<a href="https://colab.research.google.com/github/OmarEmad313/Bank-Marketing-Campaign/blob/main/Bank_Marketing_Campaign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sns
#Models:
from sklearn import svm

In [ ]:
df= pd.read_csv('bank.csv')
df

In [ ]:
#It shows the features with continuous values
df.describe()
#print(df["deposit"])

In [ ]:
# Data Exploration
print(df.head())

In [ ]:
# checcking data types
print(df.dtypes)

In [ ]:
#Data visualization
y_plot = df["deposit"].value_counts().plot(kind='bar')
print(y_plot)
#edu_plot = df["education"].value_counts().plot(kind='bar')
#print(edu_plot)
#scatter = df.plot(kind='scatter',x='age',y='balance')
#print(scatter)

In [ ]:
# Age distribution
df.age.hist()
plt.title('Histogram')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.savefig('age_distribution')

In [ ]:
# Checking feature Correlations

cor = df.corr()
num_df = df.select_dtypes(exclude="object")
corr_num = num_df.corr()
sns.heatmap(corr_num, annot=True, cmap="RdBu_r")
plt.title("Correlation Matrix", fontsize=16)
plt.show()

In [ ]:
# Encoding Target variable
target = {"deposit":     {"yes": 1, "no": 0}}
df1 = df.replace(target)
df1.head()

In [ ]:
# assignning new data set
# dropping columns with high correlation to target
df2 = df1.drop(["duration"],axis=1) #cor>0.40

In [ ]:
# Checking of missing value
count_nan = df2.isnull().sum()
print(count_nan[count_nan>0]) #no missing values

In [ ]:
#checking class balance
df.deposit.value_counts()/df.deposit.count()

In [ ]:
#I'm going to use StratifiedShuffleSplit to preserve the class proportions.
sss=StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1)
for train_index, test_index in sss.split(df.drop("deposit",axis=1), df.deposit):
    traindf=df.loc[train_index] #to select only rows (with all columns) we dont need comma and colon.
    testdf= df.loc[test_index]

In [ ]:
# Scatterplots to search for linear and non-linear relationships and histograms.
sns.pairplot(traindf, diag_kind='hist',  hue= 'deposit', height=1.5, 
             diag_kws={"edgecolor":"k", "alpha":0.5},
             plot_kws={"alpha":0.5})


In [ ]:
#Pearson’s Correlations, which measures the strength of a linear relationship
sns.heatmap(traindf.corr(method='pearson'), cmap="Greys", annot=True)

In [ ]:
#barplots showing the frequency of each category separated by label
plt.figure(figsize=[12,14])
features=["marital", "education", "contact", "default", "housing", "loan", "poutcome", "month"]
n=1
for f in features:
    plt.subplot(4,2,n)
    sns.countplot(x=f, hue='deposit', edgecolor="black", alpha=0.7, data=traindf)
    sns.despine()
    plt.title("Countplot of {}  by deposit".format(f))
    n=n+1
plt.tight_layout()
plt.show()


    
plt.figure(figsize=[14,4])
sns.countplot(x='job', hue='deposit',edgecolor="black", alpha=0.7, data=traindf)
sns.despine()
plt.title("Countplot of job by deposit")
plt.show()

In [ ]:

# Encoding Catergorical values
data = pd.get_dummies(df2, columns=["job", "marital","education","default","housing","loan","contact","month","poutcome" ])
#data.describe
data.columns

In [ ]:
# Splitting data for training and testing
#Removing the target from the data set
X = data.drop('deposit', axis=1)
#adding only the target row to y
y = data['deposit']

In [ ]:
# Create training and testing variables
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
svm_model = svm.SVC(gamma='auto')
params = {
    'C': [0.1, 1, 10],
    'gamma': [1, 0.1, 0.01],
    'kernel': ['linear','poly','rbf']
}

rf_grid = RandomizedSearchCV(
    estimator=svm_model,
    param_distributions=params,
    cv=3,
    n_jobs=4,
    verbose=2
)

rf_grid.fit(X_train, y_train)
print(rf_grid.best_params_)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


params = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

---


Result:
{'kernel': 'rbf', 'gamma': 0.001, 'C': 10}


In [ ]:
svm_pred = rf_grid.predict(X_test)
# model Evaluation
print("Accuracy:",accuracy_score(svm_pred, y_test))
# Classification Report & Confusion matrix
print(confusion_matrix(svm_pred, y_test))
print(classification_report(svm_pred, y_test))

In [ ]:
accuracy = rf_grid.best_score_ *100
print("Accuracy for our training dataset with tuning is : {:.2f}%".format(accuracy) )

In [ ]:

print('Train Accuracy:   {0:.1f}%'.format(rf_grid.score(X_train, y_train)*100))
print('Test Accuracy:    {0:.1f}%'.format(rf_grid.score(X_test, y_test)*100))